# 字符编码判断

计算机字符编码最初最原始的编码就是广为人知`ascii`编码了,不过它只能表示数字和英文字母和一些标点符号,数量非常有限,后来各个国家各种语言设定了各自的编码方式来对应自己的语言文字,而现在广泛使用`utf-8`或者`utf-16`来统一编码所有字符.因为种种历史原因,文本的编码格式非常混乱,这种时候就可以使用[chardet](http://chardet.readthedocs.io/en/latest/)工具来判别使用的是何种字符编码,这个工具我们已经在[前文](http://blog.hszofficial.site/TutorialForPython/%E6%96%87%E6%9C%AC_%E6%96%87%E4%BB%B6%E4%B8%8E%E5%AD%97%E8%8A%82%E5%BA%8F/%E6%96%87%E6%9C%AC%E5%92%8C%E7%BC%96%E7%A0%81.html#混乱的编码问题)提及


chardet可以使用pip安装.安装后可以使用命令行工具

```shell
chardetect <somefile> <someotherfile>
```

来辨别文件的字符编码类型.

## 作为模块使用

In [1]:
import requests

In [2]:
rawdata = requests.get('http://www.baike.com/wiki/%E6%9C%9F%E6%9C%9B').content

In [3]:
import chardet

In [4]:
chardet.detect(rawdata)

{'confidence': 0.99, 'encoding': 'utf-8'}

结果的`encoding`表示判断是哪种编码,`confidence`表示确信度

## 复杂情况的辨别

如果处理大量文本，您可以逐步调用检测器，当有足够的信心报告其结果，它就会停止。

In [5]:
import requests
from chardet.universaldetector import UniversalDetector

usock = requests.get('http://yahoo.co.jp/')
detector = UniversalDetector()
for line in usock.iter_lines():
    detector.feed(line)
    if detector.done: break
detector.close()
usock.close()
print(detector.result)

{'encoding': 'utf-8', 'confidence': 0.99}


如此一来我们就不需要把整个文本用于辨别,这就减少了时间


如果要检测多个文本（例如单独文件）的编码，可以重复使用单个UniversalDetector对象。
只需在每个文件的开头调用detect.reset(),调用detect.feed多次，然后调用detect.close()并检查检测器.result字典为文件的结果。

In [6]:
import glob
from chardet.universaldetector import UniversalDetector

detector = UniversalDetector()
for filename in glob.glob('src/*.py'):
    print(filename.ljust(60),end="")
    detector.reset()
    for line in open(filename, 'rb'):
        detector.feed(line)
        if detector.done: 
            break
    detector.close()
    print(detector.result)

src/gettext_te.py                                           {'encoding': 'ascii', 'confidence': 1.0}
src/international.py                                        {'encoding': 'ascii', 'confidence': 1.0}
src/pygettext.py                                            {'encoding': 'ISO-8859-2', 'confidence': 0.8550385660653095}
src/srcgettext_te.py                                        {'encoding': 'ascii', 'confidence': 1.0}
src/transfer.py                                             {'encoding': 'ascii', 'confidence': 1.0}
